In [1]:
#Importing Necessary Libraries
#%pip install webdriver_manager fake_useragent
import numpy as np
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
import csv
import random
import argparse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import os
import logging
from datetime import datetime
from fake_useragent import UserAgent
import time
import sys
import os
import sys
import json
import csv
import time
import random
import logging
import argparse
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from fake_useragent import UserAgent
import platform


In [2]:
import requests
from bs4 import BeautifulSoup
import json
import csv
import time
import random
import argparse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import os
import logging
from datetime import datetime
from fake_useragent import UserAgent

# Set up logging
log_directory = "logs"
os.makedirs(log_directory, exist_ok=True)
log_file = os.path.join(log_directory, f"menu_scraper_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class MenuScraper:
    def __init__(self, headless=True, use_proxy=False, proxy=None):
        self.use_proxy = use_proxy
        self.proxy = proxy
        self.headless = headless
        self.setup_driver()
        self.user_agent = UserAgent()
        
    def setup_driver(self):
        """Set up the Selenium WebDriver with Chrome"""
        try:
            chrome_options = Options()
            if self.headless:
                chrome_options.add_argument("--headless")
            
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument("--window-size=1920,1080")
            chrome_options.add_argument("--disable-notifications")
            chrome_options.add_argument("--disable-popup-blocking")
            chrome_options.add_argument(f"user-agent={self.user_agent.random}")
            
            if self.use_proxy and self.proxy:
                chrome_options.add_argument(f'--proxy-server={self.proxy}')
            
            service = Service(ChromeDriverManager().install())
            self.driver = webdriver.Chrome(service=service, options=chrome_options)
            logger.info("WebDriver initialized successfully")
        except Exception as e:
            logger.error(f"Error setting up WebDriver: {str(e)}")
            raise
    
    def random_delay(self, min_sec=1, max_sec=5):
        """Add a random delay to mimic human behavior and avoid detection"""
        delay = random.uniform(min_sec, max_sec)
        time.sleep(delay)
    
    def scrape_aw(self):
        """Scrape A&W menu"""
        try:
            logger.info("Starting A&W menu scraping")
            menu_url = "https://web.aw.ca/en/our-menu"
            self.driver.get(menu_url)
            
            # Wait for menu items to load
            WebDriverWait(self.driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".menu-category-container"))
            )
            
            # Allow time for dynamic content to load
            self.random_delay(2, 4)
            
            # Scroll down to load all content
            self.scroll_page()
            
            # Get page source and parse with BeautifulSoup
            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            
            menu_data = []
            
            # Extract menu categories
            menu_categories = soup.select(".menu-category-container")
            logger.info(f"Found {len(menu_categories)} menu categories")
            
            for category in menu_categories:
                # Get category name
                category_title_elem = category.select_one(".menu-category-title")
                category_name = category_title_elem.get_text(strip=True) if category_title_elem else "Uncategorized"
                logger.info(f"Processing category: {category_name}")
                
                # Extract menu items for this category
                menu_items = category.select(".menu-item")
                for item in menu_items:
                    # Extract item name
                    item_name_elem = item.select_one(".menu-item-title")
                    item_name = item_name_elem.get_text(strip=True) if item_name_elem else "Unknown"
                    
                    # Extract item description
                    item_desc_elem = item.select_one(".menu-item-desc")
                    item_description = item_desc_elem.get_text(strip=True) if item_desc_elem else ""
                    
                    # Extract price (A&W might not show prices directly on the menu page)
                    item_price_elem = item.select_one(".menu-item-price")
                    item_price = item_price_elem.get_text(strip=True) if item_price_elem else "Price not available online"
                    
                    # Extract image URL if available
                    item_image = ""
                    img_tag = item.select_one("img")
                    if img_tag and img_tag.get('src'):
                        item_image = img_tag.get('src')
                        
                        # If it's a relative URL, make it absolute
                        if item_image.startswith('/'):
                            item_image = f"https://web.aw.ca{item_image}"
                    
                    # Add nutritional info if available
                    nutritional_info = {}
                    nutrition_elem = item.select_one(".nutrition-info")
                    if nutrition_elem:
                        nutrition_items = nutrition_elem.select(".nutrition-item")
                        for n_item in nutrition_items:
                            key_elem = n_item.select_one(".nutrition-key")
                            value_elem = n_item.select_one(".nutrition-value")
                            if key_elem and value_elem:
                                key = key_elem.get_text(strip=True)
                                value = value_elem.get_text(strip=True)
                                nutritional_info[key] = value
                    
                    menu_data.append({
                        "restaurant": "A&W",
                        "category": category_name,
                        "name": item_name,
                        "price": item_price,
                        "description": item_description,
                        "image_url": item_image,
                        "nutritional_info": nutritional_info
                    })
            
            logger.info(f"Scraped {len(menu_data)} items from A&W menu")
            return menu_data
        except Exception as e:
            logger.error(f"Error scraping A&W menu: {str(e)}")
            return []
    
    def scrape_mcdonalds(self):
        """Scrape McDonald's menu"""
        try:
            logger.info("Starting McDonald's menu scraping")
            menu_url = "https://www.mcdonalds.com/us/en-us/full-menu.html"
            self.driver.get(menu_url)
            
            # Wait for menu items to load
            WebDriverWait(self.driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".category-wrapper"))
            )
            
            # Allow time for dynamic content to load
            self.random_delay(2, 4)
            
            # Scroll down to load all content
            self.scroll_page()
            
            # Get page source and parse with BeautifulSoup
            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            
            menu_data = []
            
            # Extract menu categories
            menu_categories = soup.select(".category-wrapper")
            logger.info(f"Found {len(menu_categories)} menu categories")
            
            for category in menu_categories:
                category_name = category.select_one("h2").get_text(strip=True) if category.select_one("h2") else "Uncategorized"
                logger.info(f"Processing category: {category_name}")
                
                # Extract menu items for this category
                menu_items = category.select(".cmp-category-item")
                for item in menu_items:
                    item_name = item.select_one(".item-title").get_text(strip=True) if item.select_one(".item-title") else "Unknown"
                    item_price = item.select_one(".item-price").get_text(strip=True) if item.select_one(".item-price") else "N/A"
                    item_description = item.select_one(".item-description").get_text(strip=True) if item.select_one(".item-description") else ""
                    item_image = ""
                    
                    img_tag = item.select_one("img")
                    if img_tag and img_tag.get('src'):
                        item_image = img_tag.get('src')
                    
                    menu_data.append({
                        "restaurant": "McDonald's",
                        "category": category_name,
                        "name": item_name,
                        "price": item_price,
                        "description": item_description,
                        "image_url": item_image
                    })
            
            logger.info(f"Scraped {len(menu_data)} items from McDonald's menu")
            return menu_data
        except Exception as e:
            logger.error(f"Error scraping McDonald's menu: {str(e)}")
            return []

    def scrape_burger_king(self):
        """Scrape Burger King menu"""
        try:
            logger.info("Starting Burger King menu scraping")
            menu_url = "https://www.bk.com/menu"
            self.driver.get(menu_url)
            
            # Wait for menu items to load
            WebDriverWait(self.driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".menuPage_menuCategory__Qbda1"))
            )
            
            # Allow time for dynamic content to load
            self.random_delay(2, 4)
            
            # Scroll down to load all content
            self.scroll_page()
            
            # Get page source and parse with BeautifulSoup
            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            
            menu_data = []
            
            # Extract menu categories
            menu_categories = soup.select(".menuPage_menuCategory__Qbda1")
            logger.info(f"Found {len(menu_categories)} menu categories")
            
            for category in menu_categories:
                category_name = category.select_one("h2").get_text(strip=True) if category.select_one("h2") else "Uncategorized"
                logger.info(f"Processing category: {category_name}")
                
                # Extract menu items for this category
                menu_items = category.select(".menuItem_wrapper__X_zY_")
                for item in menu_items:
                    item_name = item.select_one(".menuItem_name__on_cM").get_text(strip=True) if item.select_one(".menuItem_name__on_cM") else "Unknown"
                    item_price = item.select_one(".menuItem_price__TPsSC").get_text(strip=True) if item.select_one(".menuItem_price__TPsSC") else "N/A"
                    item_description = item.select_one(".menuItem_description__i5zkV").get_text(strip=True) if item.select_one(".menuItem_description__i5zkV") else ""
                    item_image = ""
                    
                    img_tag = item.select_one("img")
                    if img_tag and img_tag.get('src'):
                        item_image = img_tag.get('src')
                    
                    menu_data.append({
                        "restaurant": "Burger King",
                        "category": category_name,
                        "name": item_name,
                        "price": item_price,
                        "description": item_description,
                        "image_url": item_image
                    })
            
            logger.info(f"Scraped {len(menu_data)} items from Burger King menu")
            return menu_data
        except Exception as e:
            logger.error(f"Error scraping Burger King menu: {str(e)}")
            return []
    
    def scroll_page(self, pause_time=1.0):
        """Scroll down the page to ensure all dynamic content is loaded"""
        last_height = self.driver.execute_script("return document.body.scrollHeight")
        
        while True:
            # Scroll down to bottom
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            # Wait to load page
            time.sleep(pause_time)
            
            # Calculate new scroll height and compare with last scroll height
            new_height = self.driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    def save_as_json(self, data, filename):
        """Save data as JSON file"""
        try:
            with open(filename, 'w', encoding='utf-8') as json_file:
                json.dump(data, json_file, indent=4, ensure_ascii=False)
            logger.info(f"Data successfully saved to {filename}")
            return True
        except Exception as e:
            logger.error(f"Error saving JSON file: {str(e)}")
            return False
    
    def save_as_csv(self, data, filename):
        """Save data as CSV file"""
        try:
            if not data:
                logger.warning("No data to save to CSV")
                return False
                
            # For CSV, we need to flatten the nutritional_info dictionary if it exists
            flattened_data = []
            for item in data:
                flat_item = item.copy()
                
                # Handle nutritional_info if it exists
                if "nutritional_info" in flat_item and isinstance(flat_item["nutritional_info"], dict):
                    for key, value in flat_item["nutritional_info"].items():
                        flat_item[f"nutrition_{key}"] = value
                    del flat_item["nutritional_info"]
                
                flattened_data.append(flat_item)
            
            # Extract column headers from all items to ensure we include all possible fields
            all_fields = set()
            for item in flattened_data:
                all_fields.update(item.keys())
            
            fieldnames = sorted(list(all_fields))
            
            with open(filename, 'w', newline='', encoding='utf-8') as csv_file:
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(flattened_data)
            logger.info(f"Data successfully saved to {filename}")
            return True
        except Exception as e:
            logger.error(f"Error saving CSV file: {str(e)}")
            return False
    
    def scrape_restaurant(self, restaurant_name):
        """Scrape menu based on restaurant name"""
        restaurant_name = restaurant_name.lower().strip()
        
        if "a&w" in restaurant_name or "a & w" in restaurant_name:
            return self.scrape_aw()
        elif "mcdonald" in restaurant_name:
            return self.scrape_mcdonalds()
        elif "burger king" in restaurant_name or "burgerking" in restaurant_name:
            return self.scrape_burger_king()
        else:
            logger.error(f"Unsupported restaurant: {restaurant_name}")
            return []
    
    def close(self):
        """Close the WebDriver"""
        if hasattr(self, 'driver'):
            self.driver.quit()
            logger.info("WebDriver closed")

def main():
    parser = argparse.ArgumentParser(description='Scrape restaurant menus and save them in JSON and CSV formats.')
    parser.add_argument('--restaurants', nargs='+', default=["A&W", "McDonalds", "Burger King"],
                        help='List of restaurants to scrape (default: A&W, McDonalds, Burger King)')
    parser.add_argument('--output-dir', default="menu_data",
                        help='Directory to save the scraped data (default: menu_data)')
    parser.add_argument('--no-headless', action='store_true',
                        help='Run the browser in non-headless mode (visible)')
    parser.add_argument('--proxy', help='Use a proxy server (format: ip:port)')
    
    args = parser.parse_args()
    
    # Create output directory
    os.makedirs(args.output_dir, exist_ok=True)
    
    # Setup for using a proxy if specified
    use_proxy = args.proxy is not None
    
    # Initialize the scraper
    scraper = MenuScraper(
        headless=not args.no_headless,
        use_proxy=use_proxy,
        proxy=args.proxy
    )
    
    try:
        all_menu_data = []
        
        for restaurant in args.restaurants:
            logger.info(f"Processing {restaurant} menu")
            menu_data = scraper.scrape_restaurant(restaurant)
            
            if menu_data:
                # Save individual restaurant data
                restaurant_name = restaurant.lower().replace(" ", "_").replace("&", "and")
                output_base = os.path.join(args.output_dir, restaurant_name)
                
                scraper.save_as_json(menu_data, f"{output_base}_menu.json")
                scraper.save_as_csv(menu_data, f"{output_base}_menu.csv")
                
                # Add to combined data
                all_menu_data.extend(menu_data)
        
        # Save combined data if more than one restaurant was scraped
        if len(args.restaurants) > 1 and all_menu_data:
            output_base = os.path.join(args.output_dir, "all_restaurants")
            scraper.save_as_json(all_menu_data, f"{output_base}_menu.json")
            scraper.save_as_csv(all_menu_data, f"{output_base}_menu.csv")
            
    except Exception as e:
        logger.error(f"Error in main execution: {str(e)}")
    finally:
        scraper.close()

if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h]
                             [--restaurants RESTAURANTS [RESTAURANTS ...]]
                             [--output-dir OUTPUT_DIR] [--no-headless]
                             [--proxy PROXY]
ipykernel_launcher.py: error: unrecognized arguments: --f=/home/codespace/.local/share/jupyter/runtime/kernel-v312fc60deb9edae1b941be18e1510f1b849327102.json


SystemExit: 2

/home/codespace/.local/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:

# Set up logging
log_directory = 'logs'
os.makedirs(log_directory, exist_ok=True)
log_file = f"{log_directory}/menu_scraper_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class MenuScraper:
    def __init__(self, headless=True, use_proxy=False, proxy=None):
        self.headless = headless
        self.use_proxy = use_proxy
        self.proxy = proxy
        self.user_agent = UserAgent()
        self.setup_driver()
        
    def setup_driver(self):
        """Set up the Selenium WebDriver with Chrome"""
        try:
            chrome_options = Options()
            if self.headless:
                chrome_options.add_argument("--headless=new")
            
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument("--window-size=1920,1080")
            chrome_options.add_argument("--disable-gpu")
            chrome_options.add_argument("--disable-extensions")
            chrome_options.add_argument("--disable-notifications")
            chrome_options.add_argument("--disable-popup-blocking")
            chrome_options.add_argument(f"user-agent={self.user_agent.random}")
            
            if self.use_proxy and self.proxy:
                chrome_options.add_argument(f'--proxy-server={self.proxy}')
                
            # Find Chrome binary path based on operating system
            system = platform.system()
            chrome_found = False
            
            if system == "Linux":
                # First, check if Chrome is already properly installed 
                try:
                    # Try importing webdriver_manager's ChromeType for better detection
                    from webdriver_manager.core.utils import ChromeType
                    # Let webdriver_manager handle finding the Chrome binary for us
                    from webdriver_manager.chrome import ChromeDriverManager
                    
                    # No need to specify chrome_options.binary_location when using this approach
                    chrome_found = True
                except ImportError:
                    logger.info("ChromeType not available, falling back to manual detection")
                
                if not chrome_found:
                    # Common locations for Chrome binary on Linux
                    possible_paths = [
                        "/usr/bin/google-chrome",
                        "/usr/bin/google-chrome-stable",
                        "/usr/bin/chromium",
                        "/usr/bin/chromium-browser",
                        "/snap/bin/chromium",
                        "/snap/bin/google-chrome",
                    ]
                    
                    for path in possible_paths:
                        if os.path.exists(path):
                            chrome_options.binary_location = path
                            logger.info(f"Chrome binary found at: {path}")
                            chrome_found = True
                            break

            # If Chrome still not found, try to install it first
            if system == "Linux" and not chrome_found:
                logger.info("Chrome binary not found in common locations, installing Chrome...")
                try:
                    # Try different approaches to install Chrome
                    os.system("apt-get update && apt-get install -y google-chrome-stable")
                    if os.path.exists("/usr/bin/google-chrome-stable"):
                        chrome_options.binary_location = "/usr/bin/google-chrome-stable"
                        chrome_found = True
                except Exception as e:
                    logger.error(f"Error installing Chrome: {str(e)}")

            # Final fallback - try to get ChromeDriver without specifying binary location
            if not chrome_found:
                logger.warning("Could not locate Chrome binary explicitly. Trying default configuration.")
                
            # Use ChromeDriverManager to handle driver installation
            service = Service(ChromeDriverManager().install())
            self.driver = webdriver.Chrome(service=service, options=chrome_options)
            logger.info("WebDriver initialized successfully")
        except Exception as e:
            logger.error(f"Error setting up WebDriver: {str(e)}")
            raise
    
    def random_delay(self, min_sec=1, max_sec=5):
        """Add a random delay to mimic human behavior and avoid detection"""
        delay = random.uniform(min_sec, max_sec)
        time.sleep(delay)
    
    def scrape_aw(self):
        """Scrape A&W menu"""
        try:
            logger.info("Starting A&W menu scraping")
            menu_url = "https://web.aw.ca/en/our-menu"
            self.driver.get(menu_url)
            
            # Wait for menu items to load
            WebDriverWait(self.driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".menu-category-container"))
            )
            
            # Allow time for dynamic content to load
            self.random_delay(2, 4)
            
            # Scroll down to load all content
            self.scroll_page()
            
            # Get page source and parse with BeautifulSoup
            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            
            menu_data = []
            
            # Extract menu categories
            menu_categories = soup.select(".menu-category-container")
            logger.info(f"Found {len(menu_categories)} menu categories")
            
            for category in menu_categories:
                # Get category name
                category_title_elem = category.select_one(".menu-category-title")
                category_name = category_title_elem.get_text(strip=True) if category_title_elem else "Uncategorized"
                logger.info(f"Processing category: {category_name}")
                
                # Extract menu items for this category
                menu_items = category.select(".menu-item")
                for item in menu_items:
                    # Extract item name
                    item_name_elem = item.select_one(".menu-item-title")
                    item_name = item_name_elem.get_text(strip=True) if item_name_elem else "Unknown"
                    
                    # Extract item description
                    item_desc_elem = item.select_one(".menu-item-desc")
                    item_description = item_desc_elem.get_text(strip=True) if item_desc_elem else ""
                    
                    # Extract price (A&W might not show prices directly on the menu page)
                    item_price_elem = item.select_one(".menu-item-price")
                    item_price = item_price_elem.get_text(strip=True) if item_price_elem else "Price not available online"
                    
                    # Extract image URL if available
                    item_image = ""
                    img_tag = item.select_one("img")
                    if img_tag and img_tag.get('src'):
                        item_image = img_tag.get('src')
                        
                        # If it's a relative URL, make it absolute
                        if item_image.startswith('/'):
                            item_image = f"https://web.aw.ca{item_image}"
                    
                    # Add nutritional info if available
                    nutritional_info = {}
                    nutrition_elem = item.select_one(".nutrition-info")
                    if nutrition_elem:
                        nutrition_items = nutrition_elem.select(".nutrition-item")
                        for n_item in nutrition_items:
                            key_elem = n_item.select_one(".nutrition-key")
                            value_elem = n_item.select_one(".nutrition-value")
                            if key_elem and value_elem:
                                key = key_elem.get_text(strip=True)
                                value = value_elem.get_text(strip=True)
                                nutritional_info[key] = value
                    
                    menu_data.append({
                        "restaurant": "A&W",
                        "category": category_name,
                        "name": item_name,
                        "price": item_price,
                        "description": item_description,
                        "image_url": item_image,
                        "nutritional_info": nutritional_info
                    })
            
            logger.info(f"Scraped {len(menu_data)} items from A&W menu")
            return menu_data
        except Exception as e:
            logger.error(f"Error scraping A&W menu: {str(e)}")
            return []
    
    def scrape_mcdonalds(self):
        """Scrape McDonald's menu"""
        try:
            logger.info("Starting McDonald's menu scraping")
            menu_url = "https://www.mcdonalds.com/us/en-us/full-menu.html"
            self.driver.get(menu_url)
            
            # Wait for menu items to load
            WebDriverWait(self.driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".category-wrapper"))
            )
            
            # Allow time for dynamic content to load
            self.random_delay(2, 4)
            
            # Scroll down to load all content
            self.scroll_page()
            
            # Get page source and parse with BeautifulSoup
            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            
            menu_data = []
            
            # Extract menu categories
            menu_categories = soup.select(".category-wrapper")
            logger.info(f"Found {len(menu_categories)} menu categories")
            
            for category in menu_categories:
                category_name = category.select_one("h2").get_text(strip=True) if category.select_one("h2") else "Uncategorized"
                logger.info(f"Processing category: {category_name}")
                
                # Extract menu items for this category
                menu_items = category.select(".cmp-category-item")
                for item in menu_items:
                    item_name = item.select_one(".item-title").get_text(strip=True) if item.select_one(".item-title") else "Unknown"
                    item_price = item.select_one(".item-price").get_text(strip=True) if item.select_one(".item-price") else "N/A"
                    item_description = item.select_one(".item-description").get_text(strip=True) if item.select_one(".item-description") else ""
                    item_image = ""
                    
                    img_tag = item.select_one("img")
                    if img_tag and img_tag.get('src'):
                        item_image = img_tag.get('src')
                    
                    menu_data.append({
                        "restaurant": "McDonald's",
                        "category": category_name,
                        "name": item_name,
                        "price": item_price,
                        "description": item_description,
                        "image_url": item_image
                    })
            
            logger.info(f"Scraped {len(menu_data)} items from McDonald's menu")
            return menu_data
        except Exception as e:
            logger.error(f"Error scraping McDonald's menu: {str(e)}")
            return []

    def scrape_burger_king(self):
        """Scrape Burger King menu"""
        try:
            logger.info("Starting Burger King menu scraping")
            menu_url = "https://www.bk.com/menu"
            self.driver.get(menu_url)
            
            # Wait for menu items to load
            WebDriverWait(self.driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".menuPage_menuCategory__Qbda1"))
            )
            
            # Allow time for dynamic content to load
            self.random_delay(2, 4)
            
            # Scroll down to load all content
            self.scroll_page()
            
            # Get page source and parse with BeautifulSoup
            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            
            menu_data = []
            
            # Extract menu categories
            menu_categories = soup.select(".menuPage_menuCategory__Qbda1")
            logger.info(f"Found {len(menu_categories)} menu categories")
            
            for category in menu_categories:
                category_name = category.select_one("h2").get_text(strip=True) if category.select_one("h2") else "Uncategorized"
                logger.info(f"Processing category: {category_name}")
                
                # Extract menu items for this category
                menu_items = category.select(".menuItem_wrapper__X_zY_")
                for item in menu_items:
                    item_name = item.select_one(".menuItem_name__on_cM").get_text(strip=True) if item.select_one(".menuItem_name__on_cM") else "Unknown"
                    item_price = item.select_one(".menuItem_price__TPsSC").get_text(strip=True) if item.select_one(".menuItem_price__TPsSC") else "N/A"
                    item_description = item.select_one(".menuItem_description__i5zkV").get_text(strip=True) if item.select_one(".menuItem_description__i5zkV") else ""
                    item_image = ""
                    
                    img_tag = item.select_one("img")
                    if img_tag and img_tag.get('src'):
                        item_image = img_tag.get('src')
                    
                    menu_data.append({
                        "restaurant": "Burger King",
                        "category": category_name,
                        "name": item_name,
                        "price": item_price,
                        "description": item_description,
                        "image_url": item_image
                    })
            
            logger.info(f"Scraped {len(menu_data)} items from Burger King menu")
            return menu_data
        except Exception as e:
            logger.error(f"Error scraping Burger King menu: {str(e)}")
            return []
    
    def scroll_page(self, pause_time=1.0):
        """Scroll down the page to ensure all dynamic content is loaded"""
        last_height = self.driver.execute_script("return document.body.scrollHeight")
        
        while True:
            # Scroll down to bottom
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            # Wait to load page
            time.sleep(pause_time)
            
            # Calculate new scroll height and compare with last scroll height
            new_height = self.driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    def save_as_json(self, data, filename):
        """Save data as JSON file"""
        try:
            with open(filename, 'w', encoding='utf-8') as json_file:
                json.dump(data, json_file, indent=4, ensure_ascii=False)
            logger.info(f"Data successfully saved to {filename}")
            return True
        except Exception as e:
            logger.error(f"Error saving JSON file: {str(e)}")
            return False
    
    def save_as_csv(self, data, filename):
        """Save data as CSV file"""
        try:
            if not data:
                logger.warning("No data to save to CSV")
                return False
                
            # For CSV, we need to flatten the nutritional_info dictionary if it exists
            flattened_data = []
            for item in data:
                flat_item = item.copy()
                
                # Handle nutritional_info if it exists
                if "nutritional_info" in flat_item and isinstance(flat_item["nutritional_info"], dict):
                    for key, value in flat_item["nutritional_info"].items():
                        flat_item[f"nutrition_{key}"] = value
                    del flat_item["nutritional_info"]
                
                flattened_data.append(flat_item)
            
            # Extract column headers from all items to ensure we include all possible fields
            all_fields = set()
            for item in flattened_data:
                all_fields.update(item.keys())
            
            fieldnames = sorted(list(all_fields))
            
            with open(filename, 'w', newline='', encoding='utf-8') as csv_file:
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(flattened_data)
            logger.info(f"Data successfully saved to {filename}")
            return True
        except Exception as e:
            logger.error(f"Error saving CSV file: {str(e)}")
            return False
    
    def scrape_restaurant(self, restaurant_name):
        """Scrape menu based on restaurant name"""
        restaurant_name = restaurant_name.lower().strip()
        
        if "a&w" in restaurant_name or "a & w" in restaurant_name:
            return self.scrape_aw()
        elif "mcdonald" in restaurant_name:
            return self.scrape_mcdonalds()
        elif "burger king" in restaurant_name or "burgerking" in restaurant_name:
            return self.scrape_burger_king()
        else:
            logger.error(f"Unsupported restaurant: {restaurant_name}")
            return []
    
    def close(self):
        """Close the WebDriver"""
        if hasattr(self, 'driver'):
            self.driver.quit()
            logger.info("WebDriver closed")

def main():
    if 'ipykernel' in sys.argv[0]:
        restaurants = ["A&W", "McDonalds", "Burger King"]
        output_dir = "menu_data"
        headless = True
        proxy = None
        use_proxy = False

    else: 
        parser = argparse.ArgumentParser(description='Scrape restaurant menus and save them in JSON and CSV formats.')
        parser.add_argument('--restaurants', nargs='+', default=["A&W", "McDonalds", "Burger King"],
                        help='List of restaurants to scrape (default: A&W, McDonalds, Burger King)')
        parser.add_argument('--output-dir', default="menu_data",
                        help='Directory to save the scraped data (default: menu_data)')
        parser.add_argument('--no-headless', action='store_true',
                        help='Run the browser in non-headless mode (visible)')
        parser.add_argument('--proxy', help='Use a proxy server (format: ip:port)')
    
        args = parser.parse_args()
        restaurants = args.restaurants
        output_dir = args.output_dir
        headless = not args.no_headless
        proxy = args.proxy
        use_proxy = proxy is not None
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Initialize the scraper
    scraper = MenuScraper(
        headless=headless,
        use_proxy=use_proxy,
        proxy=proxy
    )
    
    try:
        all_menu_data = []
        all_menu_data = []
        
        for restaurant in restaurants:
            logger.info(f"Processing {restaurant} menu")
            menu_data = scraper.scrape_restaurant(restaurant)
            
            if menu_data:
                # Save individual restaurant data
                restaurant_name = restaurant.lower().replace(" ", "_").replace("&", "and")
                output_base = os.path.join(output_dir, restaurant_name)
                
                scraper.save_as_json(menu_data, f"{output_base}_menu.json")
                scraper.save_as_csv(menu_data, f"{output_base}_menu.csv")
                
                # Add to combined data
                all_menu_data.extend(menu_data)
        
        # Save combined data if more than one restaurant was scraped
        if len(restaurants) > 1 and all_menu_data:
            output_base = os.path.join(output_dir, "all_restaurants")
            scraper.save_as_json(all_menu_data, f"{output_base}_menu.json")
            scraper.save_as_csv(all_menu_data, f"{output_base}_menu.csv")
    except Exception as e:
        logger.error(f"Error in main execution: {str(e)}")
    finally:
        scraper.close()

if __name__ == "__main__":
    main()

2025-03-09 06:19:10,156 - INFO - ChromeType not available, falling back to manual detection
2025-03-09 06:19:10,156 - INFO - Chrome binary not found in common locations, installing Chrome...
E: List directory /var/lib/apt/lists/partial is missing. - Acquire (13: Permission denied)
2025-03-09 06:19:10,180 - WARNING - Could not locate Chrome binary explicitly. Trying default configuration.
2025-03-09 06:19:10,181 - ERROR - Error setting up WebDriver: cannot access local variable 'ChromeDriverManager' where it is not associated with a value


Reading package lists...


UnboundLocalError: cannot access local variable 'ChromeDriverManager' where it is not associated with a value

In [ ]:
# Check what files were created in the output directory
import os

def list_directory_contents(directory):
    print(f"Listing contents of: {os.path.abspath(directory)}")
    if os.path.exists(directory):
        files = os.listdir(directory)
        if files:
            for file in files:
                file_path = os.path.join(directory, file)
                file_size = os.path.getsize(file_path) / 1024  # Size in KB
                print(f"  • {file} ({file_size:.2f} KB)")
        else:
            print("  (directory is empty)")
    else:
        print(f"  Directory {directory} does not exist!")

list_directory_contents("menu_data")

Listing contents of: /workspaces/test_demo/Menu-scraping/menu_data
  Directory menu_data does not exist!
